In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import json
import pandas as pd

import tensorflow_hub as hub
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers


import platform
import time
import pathlib
import os

In [6]:
recipes=pd.read_csv('newresults.csv')

In [7]:
recipes.dropna(inplace=True)
recipes

,title,ingredients,recipe,category
0,Traditional Osso Buco,"2 pounds veal shanks, cut into short lengths ,...",Dust the veal shanks lightly with flour. Melt ...,Main Dishes
1,Italian Sausage Soup with Tortellini,"1 pound sweet Italian sausage, casings removed...","In a 5-quart Dutch oven, brown sausage. Remove...",Vegetable Soup Recipes
2,"Italian Sausage, Peppers, and Onions","6 (4 ounce) links sweet Italian sausage ,2 tab...",Place the sausage in a large skillet over medi...,Sausage
3,Tender Italian Baked Chicken,"¾ cup mayonnaise ,½ cup grated Parmesan cheese...",Preheat oven to 425 degrees F (220 degrees C)....,Italian
4,Easy Pizza Sauce I,"1 (6 ounce) can tomato paste ,1 ½ cups water ,...","Mix together the tomato paste, water, and oliv...",Pizza Sauce Recipes
...,...,...,...,...
10806,Vegetarian Pad Thai,"1 (6.75 ounce) package thin rice noodles ,2 ta...",Place noodles in a heatproof bowl and cover wi...,Thai
10807,Quick and Easy Thai Style Coleslaw,"1 (16 ounce) package shredded coleslaw mix ,¼ ...",Pour the coleslaw mix into a large bowl. Pour ...,No Mayo
10808,Sukhothai Pad Thai,"½ cup white sugar ,½ cup distilled white vineg...",To prepare Pad Thai sauce: In a medium saucepa...,Thai
10809,Slow Cook Thai Chicken,"6 skinless, boneless chicken breast halves - c...","Place the chicken breast strips, bell pepper a...",Thai


In [8]:
recipes.iloc[76
            ]

title                                               Tahini Salad
ingredients    2 pickle-size Persian cucumbers, diced ,1 larg...
recipe         Combine the cucumber, tomato, and radish in a ...
category                                  Cucumber Salad Recipes
Name: 77, dtype: object

In [9]:
title_marker='Æ Title: \n'
ingredients_marker='Ingredients: \n'
instructions_marker='Instructions: \n'

def df_to_string(recipe):
    title=recipe['title'] + '\n\n'
    ingredients=recipe['ingredients']
    instructions=recipe['recipe']
    instructions=instructions.replace('\n', '')
    
    #seperate the ingredients by comma
    ingredients=ingredients.split(',')
    instructions=instructions.split('.')
    
    #add the title marker and the title of the recipe to the return string
    return_string=title_marker + title +ingredients_marker
    
    for ingredient in ingredients:
        return_string += f'▪︎ {ingredient}\n'
        
    return_string+='\n\n'+instructions_marker
    
    for instruction in instructions:
        return_string+=f'▪︎ {instruction}.\n'
    return return_string
        
        
    

In [10]:
recipes_string=''
for i in range(recipes['title'].size):
    recipes_string+=(df_to_string(recipes.iloc[i]))



In [11]:
print(len(recipes_string))

12132467


In [12]:
def split(recipes_string, max_len, step_size = 1):

    lines = []
    next_chars = []

    for i in range(0, len(recipes_string) - max_len, step_size):
        lines.append(recipes_string[i:i+max_len])
        next_chars.append(recipes_string[i+max_len])
    
    return lines, next_chars

In [13]:
max_len = 20

lines, next_chars =  split(recipes_string, max_len = max_len, step_size = 40)
for i in range(10, 15):
    print(lines[i] + "     =>    " + next_chars[i])

ove garlic
▪︎  mince     =>    d
ated lemon zest 


I     =>    n
t the veal shanks li     =>    g
  Melt the butter in     =>     
r medium to medium-h     =>    i


In [14]:
chars = sorted(set(recipes_string))
char_indices = {char : chars.index(char) for char in chars}
X = np.zeros((len(lines), max_len, len(chars)))
y = np.zeros((len(lines), 1), dtype = np.int32)
for i, line in enumerate(lines):
	for t, char in enumerate(line):
		X[i, t, char_indices[char]] = 1
	y[i] = char_indices[next_chars[i]]

In [15]:
X.shape, y.shape

((303312, 20, 110), (303312, 1))

In [16]:
X[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
train_len = int(0.7*X.shape[0])
X_train = X[0:train_len]
X_val = X[train_len:]

y_train = y[0:train_len]
y_val  = y[train_len:]

In [18]:
model = tf.keras.models.Sequential([
    layers.LSTM(128, name = "LSTM", input_shape=(max_len, len(chars))),
    layers.Dense(len(chars))        
])

In [19]:
model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
              optimizer = "adam")

In [20]:
model.fit(X_train, 
          y_train,
          validation_data= (X_val, y_val),
          batch_size=128, epochs = 50)

Epoch 1/50
1659/1659 [==============================] - 14s 6ms/step - loss: 2.4823 - val_loss: 2.0142
Epoch 2/50
1659/1659 [==============================] - 8s 5ms/step - loss: 1.8572 - val_loss: 1.7137
Epoch 3/50
1659/1659 [==============================] - 8s 5ms/step - loss: 1.6365 - val_loss: 1.5584
Epoch 4/50
1659/1659 [==============================] - 8s 5ms/step - loss: 1.5016 - val_loss: 1.4478
Epoch 5/50
1659/1659 [==============================] - 8s 5ms/step - loss: 1.4010 - val_loss: 1.3666
Epoch 6/50
1659/1659 [==============================] - 8s 5ms/step - loss: 1.3200 - val_loss: 1.3079
Epoch 7/50
1659/1659 [==============================] - 8s 5ms/step - loss: 1.2524 - val_loss: 1.2576
Epoch 8/50
1659/1659 [==============================] - 8s 5ms/step - loss: 1.1949 - val_loss: 1.2075
Epoch 9/50
1659/1659 [==============================] - 8s 5ms/step - loss: 1.1461 - val_loss: 1.1707
Epoch 10/50
1659/1659 [==============================] - 8s 5ms/step - loss: 1.10

KeyboardInterrupt: ignored

In [21]:
def sample(preds, temp):
    
    # format the model predictions
    preds = np.asarray(preds).astype("float64")
    
    # construct normalized Boltzman with temp
    probs = np.exp(preds/temp)
    probs = probs / probs.sum()
    
    # sample from Boltzman
    samp = np.random.multinomial(1, probs, 1)
    return np.argmax(samp)

In [22]:
from scipy.special import softmax

def generate_string(seed_index, temp, gen_length, model): 
    
    # sequence of integer indices for generated text
    gen_seq = np.zeros((max_len + gen_length, len(chars)))
    
    # first part of the generated indices actually corresponds to the real text
    seed = X[seed_index]
    gen_seq[0:max_len] = seed
    
    # character version
    gen_text = lines[seed_index]
    
    # main loop. 
    # at each stage we are going to get a single 
    # character from the model prediction (with the sample function)
    # and then feed that character BACK into the model as "data"
    # for the next prediction
    for i in range(0, gen_length):
        
        # this corresponds to the part of the generated
        # text that the model can "see"
        window = gen_seq[i: i + max_len]
        
        # get the prediction and sample a single index
        preds = model.predict(np.array([window]))[0]
        preds = softmax(preds)
        next_index = sample(preds, temp)
        
        # add sampled index to the current output
        gen_seq[max_len + i, next_index] = True
        
        # create the string version
        next_char = chars[next_index]
        gen_text += next_char
    
    # only return the string version because that's what we care about
    return(gen_text)

In [23]:
gen = generate_string(8, .1, 2000, model)


In [24]:
print(gen)


▪︎  salt and pepper to taste 
▪︎ 2 %yess
▪︎  finely gCaped and gH
▪︎  X⅜ccopke︎ 
1︎ Pomges 
▪︎ 1 tablespoon #Zices tomatGer 
▪︎ ⅜ cup s.
Z︎ Heat shNetDxigh hsa garolly ToOsteY0ly3 ée3 Zicy Jack Dut 
▪︎  Cover and ™⅜t0 oi®leD’ç.
▪︎  ne¾ cuUfe⅞N VC2 ñi5 fr1iediesk
▪︎  halvho⅜
▪︎  Qu-ked pense, .

InstructionstYi miè0 äiBt*
▪︎  PMake çUsek 
▪︎ O tablespoons l°®] ûeF?y and wate’ and wacS.
▪︎  Ú⅓iBe and ?a!RBano ’aaNmovetatsN
▪︎  chopped1 quûvoZg ÚoYaPo 3Veg#ar : for [ar¼ .
▪︎ #our to°ted 1na gurDle) ¾Br®y 1Bente% .
®︎ Xouk ûold for Yoffenjor c9 half 6 to Bedium riised äacè Parmesan cheese 
▪︎ Jooss
▪︎  ︎IKedan
▪︎  : ]e⅔ked 
▪︎ ¾ cup Kotat-, a]d s%o⅔s Combr/9-in tDiÚe.
▪︎  EnUs VBret°he.
▪︎ .
Æ Title: 
v(blO⅞ Frvirf [![ing -coA¼ c¾, 9ccorO ¼ teaspoon olive oil⅛ 
▪︎ zoray" 7 tomwines #iy ®k!in8 #ench on the —a
▪︎  0ried ée/quenwBan
▪︎  beateX Remong the ; 4H︎ ; è, an(bounched yeasR reser heat 1penteKWven to⅛ to[s6 F/end &ash .
Æ Titre: oachB3 &olke.
▪' Úut KOUt shel⅛ ?etk ç▪— 3 minutes Nest

In [26]:
#!mkdir -p recipe_gen_model
model.save('recipe_gen_model.h5')

In [4]:
new_model = tf.keras.models.load_model('saved_model.pb')

OSError: ignored

In [ ]:
gen_length = 1000
seed_index = 7

for temp in [0.01, 0.02, 0.03, 0.04, 0.05]:

    gen = generate_string(seed_index, temp, gen_length, model)

    print(4*"-")
    print("TEMPERATURE: " + str(temp))
    print(gen[:-gen_length], end="")
    print(" => ", end = "")
    print(gen[-gen_length:], "")

KeyboardInterrupt: ignored